# LIBRARIES

In [1]:
import os
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp

from sklearn.model_selection import train_test_split
from sklearn import preprocessing # classification
from itertools import chain 

# FEATURE ENGINEERING
from ecgdetectors import Detectors

# CLASSIFICATION
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import keras

Using TensorFlow backend.


# CONFIGURATION

In [3]:
# ENVIRONMENT
# tf2_gpu


# physionet data
pth_data = r'C:\Users\muham\Documents\rizwan-asus\DATA\PHYSIONET-2020\data1\Training_WFDB'

# pth_code = r'C:\Users\muham\Documents\rizwan-asus\PHYSIONET2020\code\physionet-python-2020-master'

pth_functions = r'C:\Users\muham\Documents\rizwan-asus\PHYSIONET2020\code\PhysioNet_2020'

pth_eval = r'C:\Users\muham\Documents\rizwan-asus\PHYSIONET2020\results'

pth_res = r'C:\Users\muham\Documents\rizwan-asus\PHYSIONET2020\results\res1'

pth_fig = r'C:\Users\muham\Documents\rizwan-asus\PHYSIONET2020\figures'

pth_pwd = os.getcwd()

# FUNCTION

In [7]:
# # GITHUB CODE
# os.chdir(pth_code)

# from driver import *
# from get_12ECG_features import *
# from run_12ECG_classifier import *

# LOCAL FUNCTIONS
os.chdir(pth_functions)

# PHYSIONET FUNCTIONS
from driver import *
from get_12ECG_features import *
from run_12ECG_classifier import *

# RIZ FUNCTIONS
from data_read import data_files_list
from data_read import data_files_load

from data_preprocess import *
from data_prepare import *
from plt_ecg import *

os.chdir(pth_pwd)

In [8]:
def r_peaks_idx2sample(r_peaks_idx,skip_direction = 'left',skip_values =2):
    """convert r-peaks indexes to peak-peak in terms of sample"""
    # skip_values = 2
    # skip_direction = 'both' # 'left', 'right', 'both'
    
    if(skip_direction == 'left'):
        r_idx_diff = np.diff(r_peaks_idx)[skip_values:]
    elif(skip_direction == 'right'):
        r_idx_diff = np.diff(r_peaks_idx)[:-skip_values]
    elif(skip_direction == 'both'):
        r_idx_diff = np.diff(r_peaks_idx)[skip_values:-skip_values]
    else: # default - 'left'
        r_idx_diff = np.diff(r_peaks_idx)[skip_values:]
        
    return r_idx_diff

# PARAMETERS

In [9]:
sample_no = 1 # index of the data sample
lead_no = 1 # 12-lead ECG waveform (1,2,3,... 12)

TOT_LEADS = 12
OUTPUT_CLASSES = 9

ANOMALIES_REMOVAL = False
NOISE_REMOVAL = False

# LOAD DATA 

##### List of data files ```data_read.py```

In [10]:
input_files = data_files_list(pth_data)
print('Total number of input files: ',len(input_files))

print(input_files[sample_no-1])

Total number of input files:  6877
A0001.mat


#### List of data and labels  ```data_read.py```

In [11]:
[list_data,list_label,list_fname] = data_files_load(pth_data,'',False,True)

# To get only 'First Label'
list_label = [item[0] for item in list_label]

Labels from REFERENCE file


ValueError: too many values to unpack (expected 3)

In [ ]:
print('Total Samples: ',len(list_label))
label_tmp = np.array(list_label) 

print('Unique labels',len(np.unique(label_tmp))) 
del label_tmp

# DATA SPLIT 
1. Training Data: **```X_train``` & ```Y_train```**
2. Validation Data: ```X_valid``` & ```Y_valid```
3. Training Data: ```X_test``` & ```Y_test```

In [ ]:
# Split data into train and test subsets

# Train data (60%) +  Validation data (20%) + Test data (20%)
fname_train, fname_test, Y_train, Y_test = train_test_split(list_fname, list_label, test_size=0.2, shuffle=True,random_state=1)
fname_train, fname_valid, Y_train, Y_valid = train_test_split(fname_train, Y_train, test_size=0.25, shuffle=True,random_state=1)

# X_train - list of dimension samples x leads(12) x ecg signal
# Y_train - list of dimension samples x 1

In [ ]:
print(len(fname_train),len(Y_train),len(fname_valid),len(Y_valid),len(fname_test),len(Y_test))

In [12]:
print(Y_train)


NameError: name 'Y_train' is not defined

##### Extract meta-data

In [ ]:
tmp_smp_name = list_fname[sample_no-1][:-4]

print('ECG Sample Name:',tmp_smp_name)


tmp_smp_mat = os.path.join(pth_data,tmp_smp_name+'.mat')
tmp_smp_hea = os.path.join(pth_data,tmp_smp_name+'.hea')

data, header_data = load_challenge_data(tmp_smp_mat)
# data - ecg data
# header_data - contains information such as fs, gain, etc. 

In [ ]:
tmp_hea = header_data[0].split(' ')
# print(tmp_hea)
# ['A0001', '12', '500', '7500', '16-Mar-2020', '19:07:01\n']
ptID = tmp_hea[0] # 'A0001'
num_leads = int(tmp_hea[1]) # '12'
sample_Fs= int(tmp_hea[2]) # '500'
gain_lead = np.zeros(num_leads) # 1000

for ii in range(num_leads):
    tmp_hea = header_data[ii+1].split(' ')
    gain_lead[ii] = int(tmp_hea[2].split('/')[0])

In [ ]:
# for testing, we included the mean age of 57 if the age is a NaN
# This value will change as more data is being released
for iline in header_data:
    if iline.startswith('#Age'):
        tmp_age = iline.split(': ')[1].strip()
        tmp_sample_age = int(tmp_age if tmp_age != 'NaN' else 57)
    elif iline.startswith('#Sex'):
        tmp_sex = iline.split(': ')[1]
        if tmp_sex.strip()=='Female':
            tmp_sample_sex =1
        else:
            tmp_sample_sex=0
    elif iline.startswith('#Dx'):
        label = iline.split(': ')[1].split(',')[0]


In [ ]:
print(header_data)
t1 = []
t1.append(header_data)

In [ ]:
tmp_meta = np.hstack([tmp_sample_age,tmp_sample_sex,sample_Fs,gain_lead])

In [ ]:
print(tmp_meta)

In [ ]:
tmp_sample_ecg_all = data # ECG from all the leads
tmp_sample_ecg_lead = data[lead_no-1]
tmp_sample_ecg_g = gain_lead[lead_no-1]
tmp_sample_ecg_fs = sample_Fs

In [ ]:
print('Sample (length): ',len(tmp_sample_ecg_lead))
print('Sample (label): ',list_label[sample_no-1])
print('Sample (gain): ',tmp_sample_ecg_g)
print('Sample (sampling-frequency): ',tmp_sample_ecg_fs)

In [ ]:
print('Sample - All (shape): ',np.shape(tmp_sample_ecg_all))
print('Sample - Lead (shape): ',np.shape(tmp_sample_ecg_lead))

# EXPLORATORY DATA ANALYSIS

### Plot ECG Waveform

__Parameters__

- ```sample_no```
- ```lead_no```

#### Single Lead    ```lead_no```

In [ ]:
cm = plt.get_cmap('gist_rainbow')
fig = plt.figure()

plt.plot(list_data[sample_no-1][lead_no-1,:],color='b',linestyle = '-' )

plt.xticks(fontsize = 12)
plt.yticks(fontsize = 12)


tmp_pth = os.path.join(pth_fig,'raw_single_lead',list_fname[sample_no-1][:-4])

print('Raw figure path: ',tmp_pth)
plt.savefig(tmp_pth+'_raw_l'+str(lead_no)+'.png',dpi = 300)

#### Multiple Lead 

In [ ]:
NUM_COLORS = 12 # Total Leads = 12
LINE_STYLES = ['solid', 'dashed', 'dashdot', 'dotted']
NUM_STYLES = len(LINE_STYLES)

legend_lst = []
cm = plt.get_cmap('gist_rainbow')
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(NUM_COLORS):
    lines = ax.plot(list_data[sample_no-1][i,:])
    lines[0].set_color(cm(i//NUM_STYLES*float(NUM_STYLES)/NUM_COLORS))
    lines[0].set_linestyle(LINE_STYLES[i%NUM_STYLES])
    legend_lst.append('L'+str(i+1))

ax.legend(legend_lst,loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True, ncol=3)
tmp_pth = os.path.join(pth_fig,'raw_multiple_leads',list_fname[sample_no-1][:-4])
fig.savefig(tmp_pth+'_raw_multiple.png',dpi = 300,bbox_inches='tight')
plt.show()

# DATA PREPARATION

### @vector

- ```TO DO```

# PRE-PROCESSING

#### Type I
- removal of anomalies
- removal of noise

In [ ]:
if(ANOMALIES_REMOVAL):
    print('ANOMALIES REMOVAL - DONE')
    tmp_sample_sig = preprocess_type1(tmp_sample_ecg_all)
    print(np.shape(tmp_sample_sig))
    tmp_sample_preprocess = tmp_sample_sig[0,0,:,0]

In [ ]:
if(NOISE_REMOVAL):
    print('To Do')

# FEATURE ENGINEERING

### R-peaks: ```physionet```

In [ ]:

r_peaks_pnet,r_idx_pnet = detect_peaks(tmp_sample_ecg_lead,tmp_sample_ecg_fs,tmp_sample_ecg_g)
# r_peaks_pnet - peak values based on physionet algorithm
# r_idx_pnet - peak indices based on physionet algorithm

r_peaks_pnet = r_peaks_pnet.astype(int)
r_idx_pnet_sample = r_peaks_idx2sample(r_idx_pnet)

In [ ]:
print(r_peaks_pnet)
print(r_idx_pnet)

print(len(r_peaks_pnet))
print(len(r_idx_pnet))

In [ ]:
#   mean
mean_RR = np.mean(r_idx_pnet/tmp_sample_ecg_fs*1000)
mean_Peaks = np.mean(r_peaks_pnet)

print(mean_RR)
print

### R-peaks: ```py-ecg-detector```

In [ ]:
detectors = Detectors(tmp_sample_ecg_fs)

tmp_ecg_integ = ecg_sig_integrated(tmp_sample_ecg_lead,tmp_sample_ecg_fs)
tmp_input_features = tmp_sample_ecg_lead


# Hamilton
r_idx_hamilton = detectors.hamilton_detector(tmp_input_features) 
r_idx_hamilton_sample = r_peaks_idx2sample(r_idx_hamilton)
# r_peaks_hamilton = tmp_ecg_integ[r_idx_hamilton]

# Christov
r_idx_christov = detectors.christov_detector(tmp_input_features)
# r_peaks_christov = tmp_ecg_integ[r_idx_christov]

# Engelse & Zeelenberg
r_idx_engelse = detectors.engzee_detector(tmp_input_features)
# r_peaks_engelse = tmp_ecg_integ[r_idx_engelse]

# Pan & Tompkins
r_idx_pan = detectors.pan_tompkins_detector(tmp_input_features)
# r_peaks_pan = tmp_ecg_integ[r_idx_pan]

# Stationary Wavelet Transform
r_idx_wavelet = detectors.swt_detector(tmp_input_features)
# r_peaks_wavelet = tmp_ecg_integ[r_idx_wavelet]

#Two Moving Average
r_idx_mavg = detectors.two_average_detector(tmp_input_features)
# r_peaks_mavg = tmp_ecg_integ[r_idx_mavg]

# Matched Filter
# r_peaks_mfilter = detectors.matched_filter_detector(tmp_sample_x)

### ```pyhrv```

### ```heartpy```

#### ```WIP```

- matplotlib==2.2.2
- ipywidgets==7.2.1
- scipy==1.0.1
- numpy==1.14.2
- pandas==0.22.0
- biosppy
- pyentrp
- pywt

In [ ]:
# Import local Libraries
sys.path.insert(0, os.path.dirname(os.getcwd()))
from features.feature_extractor import Features
from ecg_features.utils.plotting.waveforms import plot_waveforms

#### Heartrate Variability Analysis

### Plot - ```r-peak```

In [ ]:
R_PEAK_DETECTION = 'pan'
# 'pnet','hamilton', 'christov', 'engelse', 'pan','wavelet', 'mavg'

In [ ]:
plt.figure()
plt.plot(tmp_input_features)
if(R_PEAK_DETECTION == 'pnet'): 
    plt.plot(r_peaks_pnet, tmp_input_features[r_peaks_hamilton], 'ro')
elif(R_PEAK_DETECTION == 'hamilton'): 
    plt.plot(r_peaks_hamilton, tmp_input_features[r_peaks_hamilton], 'ro')
elif(R_PEAK_DETECTION == 'christov'): 
    plt.plot(r_peaks_christov, tmp_input_features[r_peaks_christov], 'ro')
elif(R_PEAK_DETECTION == 'engelse'): 
    plt.plot(r_peaks_engelse, tmp_input_features[r_peaks_engelse], 'ro')
elif(R_PEAK_DETECTION == 'pan'): 
    plt.plot(r_idx_pan, tmp_input_features[r_idx_pan], 'ro')
elif(R_PEAK_DETECTION == 'wavelet'): 
    plt.plot(r_peaks_wavelet, tmp_input_features[r_peaks_wavelet], 'ro')
elif(R_PEAK_DETECTION == 'mavg'): 
    plt.plot(r_peaks_mavg, tmp_input_features[r_peaks_mavg], 'ro')
    
plt.title('Detected R-peaks')
# plt.show()

tmp_pth = os.path.join(pth_fig,list_fname[sample_no-1][:-4]+'_lead'+str(lead_no)+'_rpeak_'+R_PEAK_DETECTION)

print('Raw figure path: ',tmp_pth)
plt.savefig(tmp_pth+'.png',dpi = 300)

### FEATURE MATRIX

In [ ]:
#   mean
mean_RR = np.mean(r_idx_pnet_sample/tmp_sample_ecg_fs)
mean_Peaks = np.mean(r_peaks_pnet*tmp_sample_ecg_g)

#   median
median_RR = np.median(r_idx_pnet_sample/tmp_sample_ecg_fs)
median_Peaks = np.median(r_peaks_pnet*tmp_sample_ecg_g)

#   standard deviation
std_RR = np.std(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
std_Peaks = np.std(r_peaks_pnet*tmp_sample_ecg_g)

#   variance
var_RR = stats.tvar(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
var_Peaks = stats.tvar(r_peaks_pnet*tmp_sample_ecg_g)

#   Skewness
skew_RR = stats.skew(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
skew_Peaks = stats.skew(r_peaks_pnet*tmp_sample_ecg_g)

#   Kurtosis
kurt_RR = stats.kurtosis(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
kurt_Peaks = stats.kurtosis(r_peaks_pnet*tmp_sample_ecg_g)


In [ ]:
features_samp = np.hstack([tmp_sample_age,tmp_sample_sex,mean_RR,mean_Peaks,median_RR,median_Peaks,std_RR,std_Peaks,var_RR,var_Peaks,skew_RR,skew_Peaks,kurt_RR,kurt_Peaks])

In [ ]:
print(np.shape(features_samp))

In [ ]:
print(features_samp)

# CLASSIFICATION

#### TEMP: Backup of variables

In [ ]:
Y_train_bup = Y_train.copy()
X_train_bup = X_train.copy()

Y_valid_bup = Y_valid.copy()
X_valid_bup = X_valid.copy()

Y_test_bup = Y_test.copy()
X_test_bup = X_test.copy()

In [ ]:
print(len(X_train))
print(len(Y_train))

In [ ]:
features_matrix = []
print('FEATURE TYPE = raw-data')
lead_no = 1
NO_SAMPLES = 4500
for ii in range(len(list_fname)):
    #-------------------------------------------------
    # META DATA FEATURES
    #-------------------------------------------------
    tmp_smp_name = list_fname[ii][:-4]

    print('ECG Sample Name:' ,ii,tmp_smp_name)


    tmp_smp_mat = os.path.join(pth_data,tmp_smp_name+'.mat')
    tmp_smp_hea = os.path.join(pth_data,tmp_smp_name+'.hea')

    data, header_data = load_challenge_data(tmp_smp_mat)
    # data - ecg data
    # header_data - contains information such as fs, gain, etc.

    tmp_sample_ecg_all = data # ECG from all the leads
    tmp_sample_ecg_lead = data[lead_no-1]

    features_samp = np.zeros((0, NO_SAMPLES))


    if(len(tmp_sample_ecg_lead) > NO_SAMPLES):
        features_samp = tmp_sample_ecg_lead[0:NO_SAMPLES]
    else:
        features_samp[0,0:len(tmp_sample_ecg_lead)] = tmp_sample_ecg_lead

    features_matrix.append(features_samp)

    del features_samp


In [ ]:
print(len(features_matrix))
print(len(fname_test))

In [ ]:
def ecg_feature_extract(pth_data, list_fname, feat_type):
    features_matrix = []
    #for ii in range(len(list_data)):
    
    if(feat_type == 'raw-data'):
        print('FEATURE TYPE = raw-data')
        lead_no = 1
        NO_SAMPLES = 4500
        for ii in range(len(list_fname)):
            #-------------------------------------------------
            # META DATA FEATURES
            #-------------------------------------------------
            tmp_smp_name = list_fname[ii][:-4]

            print('ECG Sample Name:',tmp_smp_name)


            tmp_smp_mat = os.path.join(pth_data,tmp_smp_name+'.mat')
            tmp_smp_hea = os.path.join(pth_data,tmp_smp_name+'.hea')

            data, header_data = load_challenge_data(tmp_smp_mat)
            # data - ecg data
            # header_data - contains information such as fs, gain, etc.
            
            tmp_sample_ecg_all = data # ECG from all the leads
            tmp_sample_ecg_lead = data[lead_no-1]

            features_samp = np.zeros((0, NO_SAMPLES))

            
            if(len(tmp_sample_ecg_lead) > NO_SAMPLES):
                features_samp = tmp_sample_ecg_lead[0:NO_SAMPLES]
            else:
                features_samp[0,0:len(tmp_sample_ecg_lead)] = tmp_sample_ecg_lead
                    
            features_matrix.append(features_samp)

            del features_samp
            
        return np.asarray(features_matrix)
            
    
    else:
        lead_no = 1
        for ii in range(len(list_fname)):
            #-------------------------------------------------
            # META DATA FEATURES
            #-------------------------------------------------
            tmp_smp_name = list_fname[ii][:-4]

            print('ECG Sample Name:',tmp_smp_name)


            tmp_smp_mat = os.path.join(pth_data,tmp_smp_name+'.mat')
            tmp_smp_hea = os.path.join(pth_data,tmp_smp_name+'.hea')

            data, header_data = load_challenge_data(tmp_smp_mat)
            # data - ecg data
            # header_data - contains information such as fs, gain, etc. 

            tmp_hea = header_data[0].split(' ')
            # print(tmp_hea)
            # ['A0001', '12', '500', '7500', '16-Mar-2020', '19:07:01\n']
            ptID = tmp_hea[0] # 'A0001'
            num_leads = int(tmp_hea[1]) # '12'
            sample_Fs= int(tmp_hea[2]) # '500'
            gain_lead = np.zeros(num_leads) # 1000

            for ii in range(num_leads):
                tmp_hea = header_data[ii+1].split(' ')
                gain_lead[ii] = int(tmp_hea[2].split('/')[0])

            # for testing, we included the mean age of 57 if the age is a NaN
            # This value will change as more data is being released
            for iline in header_data:
                if iline.startswith('#Age'):
                    tmp_age = iline.split(': ')[1].strip()
                    tmp_sample_age = int(tmp_age if tmp_age != 'NaN' else 57)
                elif iline.startswith('#Sex'):
                    tmp_sex = iline.split(': ')[1]
                    if tmp_sex.strip()=='Female':
                        tmp_sample_sex =1
                    else:
                        tmp_sample_sex=0
                elif iline.startswith('#Dx'):
                    label = iline.split(': ')[1].split(',')[0]





            tmp_sample_ecg_all = data # ECG from all the leads
            tmp_sample_ecg_lead = data[lead_no-1]
            tmp_sample_ecg_g = gain_lead[lead_no-1]
            tmp_sample_ecg_fs = sample_Fs
            #------------------------------------------------------------
            # R-Peaks Features
            #------------------------------------------------------------
            r_peaks_pnet,r_idx_pnet = detect_peaks(tmp_sample_ecg_lead,tmp_sample_ecg_fs,tmp_sample_ecg_g)

            r_peaks_pnet = r_peaks_pnet.astype(int)
            r_idx_pnet_sample = r_peaks_idx2sample(r_idx_pnet)


            #------------------------------------------------------------
            # R-Peaks Statistical Features
            #------------------------------------------------------------
            #   mean
            mean_RR = np.mean(r_idx_pnet_sample/tmp_sample_ecg_fs)
            mean_Peaks = np.mean(r_peaks_pnet*tmp_sample_ecg_g)

            #   median
            median_RR = np.median(r_idx_pnet_sample/tmp_sample_ecg_fs)
            median_Peaks = np.median(r_peaks_pnet*tmp_sample_ecg_g)

            #   standard deviation
            std_RR = np.std(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
            std_Peaks = np.std(r_peaks_pnet*tmp_sample_ecg_g)

            #   variance
            var_RR = stats.tvar(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
            var_Peaks = stats.tvar(r_peaks_pnet*tmp_sample_ecg_g)

            #   Skewness
            skew_RR = stats.skew(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
            skew_Peaks = stats.skew(r_peaks_pnet*tmp_sample_ecg_g)

            #   Kurtosis
            kurt_RR = stats.kurtosis(r_idx_pnet_sample/tmp_sample_ecg_fs*1000)
            kurt_Peaks = stats.kurtosis(r_peaks_pnet*tmp_sample_ecg_g)

            features_samp = np.hstack([tmp_sample_age,tmp_sample_sex,mean_RR,mean_Peaks,median_RR,median_Peaks,std_RR,std_Peaks,var_RR,var_Peaks,skew_RR,skew_Peaks,kurt_RR,kurt_Peaks])

            features_matrix.append(features_samp)

            del features_samp

        return np.asarray(features_matrix)


In [ ]:
print(np.shape(X_train))

In [ ]:
X_train = ecg_feature_extract(pth_data, fname_train,'raw-data')
X_valid = ecg_feature_extract(pth_data, fname_valid,'raw-data')
X_test = ecg_feature_extract(pth_data, fname_test,'raw-data')

### Data preparation for classification

In [ ]:
# Data labels into matrix form i.e. [no of samples x no of output classes]
lb = preprocessing.LabelBinarizer()
lb.fit(Y_train)
# lb.classes_
Y_train = lb.transform(Y_train)
Y_valid = lb.transform(Y_valid)
Y_test = lb.transform(Y_test)

In [ ]:
print(np.shape(X_train))

In [ ]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_valid = np.reshape(X_valid, (X_valid.shape[0], 1, X_valid.shape[1]))
X_test = np.reshape(X_valid, (X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
X_train.shape[1]

### LSTM Model

In [ ]:
feat_dim = X_train.shape[2]

In [ ]:
# create and fit the LSTM network
batch_size = 64
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(1, feat_dim)))
#model.add(Dropout(0.25))
model.add(LSTM(256, return_sequences=True))
#model.add(Dropout(0.25))
model.add(LSTM(128, return_sequences=True))
#model.add(Dropout(0.25))
model.add(LSTM(64, return_sequences=True))
#model.add(Dropout(0.25))
model.add(LSTM(32))
model.add(Dense(OUTPUT_CLASSES, activation='softmax'))
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=50, verbose=1, mode='auto')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=100, batch_size=batch_size, validation_data=(X_valid, Y_valid), verbose=2, shuffle=False, callbacks=[early_stopping])


In [ ]:
model.save('ecg_lstm.h5')

In [ ]:
pred_prob = model.predict(X_test)
pred_classes = model.predict_classes(X_test)

In [ ]:
score = accuracy_score(Y_test, lb.transform(pred_classes))
print(score)

### LSTM Model

In [ ]:
from keras.layers import (Input, Conv1D, MaxPooling1D, Dropout,
                          BatchNormalization, Activation, Add,
                          Flatten, Dense)
from keras.models import Model

In [ ]:
class ResidualUnit(object):
    """Residual unit block (unidimensional).
    Parameters
    ----------
    n_samples_out: int
        Number of output samples.
    n_filters_out: int
        Number of output filters.
    kernel_initializer: str, otional
        Initializer for the weights matrices. See Keras initializers. By default it uses
        'he_normal'.
    dropout_rate: float [0, 1), optional
        Dropout rate used in all Dropout layers. Default is 0.8
    kernel_size: int, optional
        Kernel size for convolutional layers. Default is 17.
    preactivation: bool, optional
        When preactivation is true use full preactivation architecture proposed
        in [1]. Otherwise, use architecture proposed in the original ResNet
        paper [2]. By default it is true.
    postactivation_bn: bool, optional
        Defines if you use batch normalization before or after the activation layer (there
        seems to be some advantages in some cases:
        https://github.com/ducha-aiki/caffenet-benchmark/blob/master/batchnorm.md).
        If true, the batch normalization is used before the activation
        function, otherwise the activation comes first, as it is usually done.
        By default it is false.
    activation_function: string, optional
        Keras activation function to be used. By default 'relu'.
    References
    ----------
    .. [1] K. He, X. Zhang, S. Ren, and J. Sun, "Identity Mappings in Deep Residual Networks,"
           arXiv:1603.05027 [cs], Mar. 2016. https://arxiv.org/pdf/1603.05027.pdf.
    .. [2] K. He, X. Zhang, S. Ren, and J. Sun, "Deep Residual Learning for Image Recognition," in 2016 IEEE Conference
           on Computer Vision and Pattern Recognition (CVPR), 2016, pp. 770-778. https://arxiv.org/pdf/1512.03385.pdf
    """

    def __init__(self, n_samples_out, n_filters_out, kernel_initializer='he_normal',
                 dropout_rate=0.8, kernel_size=17, preactivation=True,
                 postactivation_bn=False, activation_function='relu'):
        self.n_samples_out = n_samples_out
        self.n_filters_out = n_filters_out
        self.kernel_initializer = kernel_initializer
        self.dropout_rate = dropout_rate
        self.kernel_size = kernel_size
        self.preactivation = preactivation
        self.postactivation_bn = postactivation_bn
        self.activation_function = activation_function

    def _skip_connection(self, y, downsample, n_filters_in):
        """Implement skip connection."""
        # Deal with downsampling
        if downsample > 1:
            y = MaxPooling1D(downsample, strides=downsample, padding='same')(y)
        elif downsample == 1:
            y = y
        else:
            raise ValueError("Number of samples should always decrease.")
        # Deal with n_filters dimension increase
        if n_filters_in != self.n_filters_out:
            # This is one of the two alternatives presented in ResNet paper
            # Other option is to just fill the matrix with zeros.
            y = Conv1D(self.n_filters_out, 1, padding='same',
                       use_bias=False, kernel_initializer=self.kernel_initializer)(y)
        return y

    def _batch_norm_plus_activation(self, x):
        if self.postactivation_bn:
            x = Activation(self.activation_function)(x)
            x = BatchNormalization(center=False, scale=False)(x)
        else:
            x = BatchNormalization()(x)
            x = Activation(self.activation_function)(x)
        return x

    def __call__(self, inputs):
        """Residual unit."""
        x, y = inputs
        n_samples_in = y.shape[1].value
        downsample = n_samples_in // self.n_samples_out
        n_filters_in = y.shape[2].value
        y = self._skip_connection(y, downsample, n_filters_in)
        # 1st layer
        x = Conv1D(self.n_filters_out, self.kernel_size, padding='same',
                   use_bias=False, kernel_initializer=self.kernel_initializer)(x)
        x = self._batch_norm_plus_activation(x)
        if self.dropout_rate > 0:
            x = Dropout(self.dropout_rate)(x)

        # 2nd layer
        x = Conv1D(self.n_filters_out, self.kernel_size, strides=downsample,
                   padding='same', use_bias=False,
                   kernel_initializer=self.kernel_initializer)(x)
        if self.preactivation:
            x = Add()([x, y])  # Sum skip connection and main connection
            y = x
            x = self._batch_norm_plus_activation(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
        else:
            x = BatchNormalization()(x)
            x = Add()([x, y])  # Sum skip connection and main connection
            x = Activation(self.activation_function)(x)
            if self.dropout_rate > 0:
                x = Dropout(self.dropout_rate)(x)
            y = x
        return [x, y]

In [ ]:
# ----- Model ----- #
kernel_size = 16
kernel_initializer = 'he_normal'
signal = Input(shape=(4096, 12), dtype=np.float32, name='signal')
age_range = Input(shape=(6,), dtype=np.float32, name='age_range')
is_male = Input(shape=(1,), dtype=np.float32, name='is_male')


x = signal
x = Conv1D(64, kernel_size, padding='same', use_bias=False,
           kernel_initializer=kernel_initializer)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x, y = ResidualUnit(1024, 128, kernel_size=kernel_size,
                    kernel_initializer=kernel_initializer)([x, x])
x, y = ResidualUnit(256, 196, kernel_size=kernel_size,
                    kernel_initializer=kernel_initializer)([x, y])
x, y = ResidualUnit(64, 256, kernel_size=kernel_size,
                    kernel_initializer=kernel_initializer)([x, y])
x, _ = ResidualUnit(16, 320, kernel_size=kernel_size,
                    kernel_initializer=kernel_initializer)([x, y])
x = Flatten()(x)
diagn = Dense(6, activation='sigmoid', kernel_initializer=kernel_initializer)(x)
model = Model(signal, diagn)

In [ ]:
history = model.fit(x, y,
            batch_size=batch_size,
            epochs=70,
            initial_epoch=0,  # If you are continuing a interrupted section change here
            validation_split=args.val_split,
            shuffle='batch',  # Because our dataset is an HDF5 file
            callbacks=callbacks,
            verbose=1)

### MISC

# JUNK